In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve

In [2]:
grouped_purchased = pd.read_csv('cvs_data_file/online-retail.csv', header = 0)
grouped_purchased.head()

,CustomerID,StockCode,Quantity
0,12346,23166,1
1,12347,16008,24
2,12347,17021,36
3,12347,20665,6
4,12347,20719,40


In [3]:
customers = list(np.sort(grouped_purchased.CustomerID.unique())) # Get our unique customers
products = list(grouped_purchased.StockCode.unique()) # Get our unique products that were purchased
quantity = list(grouped_purchased.Quantity) # All of our purchases

rows = grouped_purchased.CustomerID.astype('category', categories = customers).cat.codes 
# Get the associated row indices
cols = grouped_purchased.StockCode.astype('category', categories = products).cat.codes 
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  import sys


In [4]:
purchases_sparse

<4338x3664 sparse matrix of type '<class 'numpy.int32'>'
	with 266723 stored elements in Compressed Sparse Row format>

In [5]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

98.32190920694744

In [6]:
import random

In [7]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

In [8]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

In [9]:
product_train

<4338x3664 sparse matrix of type '<class 'numpy.int32'>'
	with 213378 stored elements in Compressed Sparse Row format>

In [10]:
import implicit

In [11]:
alpha = 15
user_vecs, item_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

This method is deprecated. Please use the AlternatingLeastSquares class instead
100%|██████████████████████████████████████████████████████████| 50.0/50 [00:03<00:00, 12.63it/s]


In [12]:
user_vecs, item_vecs

(array([[ 3.3310715e-03, -5.0435256e-04, -1.0577244e-03, ...,
         -1.0687101e-03, -5.8253354e-04,  3.6102749e-04],
        [-1.6332516e-02,  2.7456803e-02, -1.0156110e-02, ...,
          5.1277275e-03, -3.1941090e-02,  1.2390122e-02],
        [-1.4519610e-02,  9.2483498e-03,  2.5906937e-02, ...,
         -1.3467503e-02, -7.8241806e-04, -3.2888756e-03],
        ...,
        [ 2.6828764e-02,  1.9564910e-02,  1.0152447e-02, ...,
         -2.8459286e-05,  1.3128698e-02, -1.1472919e-03],
        [ 8.7323063e-04,  2.5040754e-03,  1.4163433e-02, ...,
          1.0711586e-02,  1.4322254e-02, -2.4558625e-03],
        [-1.0377470e-02,  3.8644816e-03,  1.5555532e-02, ...,
          3.3192404e-02,  4.8777233e-03, -4.1844249e-02]], dtype=float32),
 array([[15.701957  ,  3.6738596 ,  1.2833034 , ...,  1.113787  ,
          2.6994147 ,  5.0555353 ],
        [ 4.3682714 ,  8.061658  , 12.384979  , ...,  4.648796  ,
         -0.02420726,  1.4152055 ],
        [ 7.688288  ,  4.4307284 ,  5.509299  

In [13]:
from sklearn import metrics

In [14]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)   

In [15]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [16]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

(0.87, 0.814)

In [17]:
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

In [18]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    products_list - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    return item_lookup.loc[item_lookup.StockCode.isin(prod_codes)]

In [19]:
customers_arr[:5]

array([12346, 12347, 12348, 12349, 12350], dtype=int64)

In [20]:
item_lookup = pd.read_csv('cvs_data_file/item_lookup.csv', header = 0)
item_lookup.head()

,StockCode,Description
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,71053,WHITE METAL LANTERN
2,84406B,CREAM CUPID HEARTS COAT HANGER
3,84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,84029E,RED WOOLLY HOTTIE WHITE HEART.


In [21]:
get_items_purchased(12346, product_train, customers_arr, products_arr, item_lookup)

,StockCode,Description
2592,23166,MEDIUM CERAMIC TOP STORAGE JAR


In [22]:
from sklearn.preprocessing import MinMaxScaler

In [23]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_items - The number of items you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the user/item vectors for items never interacted with/purchased
    '''
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, item_lookup.Description.loc[item_lookup.StockCode == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'StockCode': codes, 'Description': descriptions}) # Create a dataframe 
    return final_frame[['StockCode', 'Description']] # Switch order of columns around

In [24]:
rec_items(12346, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

,StockCode,Description
0,23167,SMALL CERAMIC TOP STORAGE JAR
1,22963,JAM JAR WITH GREEN LID
2,23165,LARGE CERAMIC TOP STORAGE JAR
3,22962,JAM JAR WITH PINK LID
4,22980,PANTRY SCRUBBING BRUSH
5,22982,PANTRY PASTRY BRUSH
6,22979,PANTRY WASHING UP BRUSH
7,23295,SET OF 12 MINI LOAF BAKING CASES
8,22978,PANTRY ROLLING PIN
9,22965,3 TRADITIONAL COOKIE CUTTERS SET


In [25]:
get_items_purchased(12353, product_train, customers_arr, products_arr, item_lookup)

,StockCode,Description
827,37446,MINI CAKE STAND WITH HANGING CAKES
828,37449,CERAMIC CAKE STAND + HANGING CAKES
1279,37450,CERAMIC CAKE BOWL + HANGING CAKES
1312,22890,NOVELTY BISCUITS CAKE STAND 3 TIER


In [26]:
rec_items(12353, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup)

,StockCode,Description
0,22055,MINI CAKE STAND HANGING STRAWBERY
1,21231,SWEETHEART CERAMIC TRINKET BOX
2,22059,CERAMIC STRAWBERRY DESIGN MUG
3,21232,STRAWBERRY CERAMIC TRINKET BOX
4,22057,CERAMIC PLATE STRAWBERRY DESIGN
5,37448,CERAMIC CAKE DESIGN SPOTTED MUG
6,22644,CERAMIC CHERRY CAKE MONEY BANK
7,37447,CERAMIC CAKE DESIGN SPOTTED PLATE
8,22063,CERAMIC BOWL WITH STRAWBERRY DESIGN
9,22649,STRAWBERRY FAIRY CAKE TEAPOT


In [27]:
get_items_purchased(12361, product_train, customers_arr, products_arr, item_lookup)

,StockCode,Description
34,22326,ROUND SNACK BOXES SET OF4 WOODLAND
35,22629,SPACEBOY LUNCH BOX
37,22631,CIRCUS PARADE LUNCH BOX
66,20725,LUNCH BAG RED RETROSPOT
265,22382,LUNCH BAG SPACEBOY DESIGN
362,22328,ROUND SNACK BOXES SET OF 4 FRUITS
364,22630,DOLLY GIRL LUNCH BOX
691,22555,PLASTERS IN TIN STRONGMAN
2566,20725,LUNCH BAG RED SPOTTY


In [30]:
rec_items(12361, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

,StockCode,Description
0,22662,LUNCH BAG DOLLY GIRL DESIGN
1,22551,PLASTERS IN TIN SPACEBOY
2,20726,LUNCH BAG WOODLAND
3,22554,PLASTERS IN TIN WOODLAND ANIMALS
4,22556,PLASTERS IN TIN CIRCUS PARADE
5,20719,WOODLAND CHARLOTTE BAG
6,22614,PACK OF 12 SPACEBOY TISSUES
7,22383,LUNCH BAG SUKI DESIGN
8,20727,LUNCH BAG BLACK SKULL.
9,23206,LUNCH BAG APPLE DESIGN


In [50]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k,auc_score


# Instantiate and train the model
model = LightFM(loss='warp')
model.fit(product_train, epochs=40, num_threads=2)

# Evaluate the trained model
test_precision = precision_at_k(model,product_test, k=5).mean()
train_precision = precision_at_k(model, product_train, k=10).mean()

train_auc = auc_score(model, product_train).mean()
test_auc = auc_score(model, product_test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))


Precision: train 0.27, test 0.35.
AUC: train 0.91, test 0.90.


In [90]:
def display_recommended_items(model, data, user_ids):

    print(data.shape)
    
    n_users, n_items = data.shape

    for id in user_ids:

        known_positives = item_lookup['Description'][product_train.tocsr()[id].indices]
        known_positives_df = pd.DataFrame(data=known_positives)
        print(known_positives_df,'\n','######################################')            
       
        scores = model.predict(id, np.arange(n_items))

        top_items = item_lookup['Description'][np.argsort(-scores)]
        df = pd.DataFrame(data=top_items)
        print(df)            
            

display_recommended_items(model, product_train, [3664])


(4338, 3664)
                              Description
314    LOVEBIRD HANGING DECORATION WHITE 
396     RIBBON REEL CHRISTMAS SOCK BAUBLE
450       CERAMIC PIRATE CHEST MONEY BANK
625   CREAM SLICE FLANNEL CHOCOLATE SPOT 
825    MAGNETS PACK OF 4 CHILDHOOD MEMORY
831                OFFICE MUG WARMER PINK
844    WOOD STOCKING CHRISTMAS SCANDISPOT
845         SET OF 4 NAPKIN CHARMS HEARTS
881                    FINE WICKER HEART 
1072      CREAM WALL PLANTER HEART SHAPED
1131             ARMY CAMO BOOKCOVER TAPE
1204                    PINK OWL SOFT TOY
1334   3 HEARTS HANGING DECORATION RUSTIC
1391      PINK VINTAGE PAISLEY PICNIC BAG
1563              PINK GLASS CANDLEHOLDER
1583                       SANDALWOOD FAN
1718             ABSTRACT CIRCLE JOURNAL 
1917   TROPICAL  HONEYCOMB PAPER GARLAND 
2183        PURPLE ACRYLIC FACETED BANGLE
2476          DAISIES  HONEYCOMB GARLAND 
2574                        BUNNY EGG BOX
2720      BLACK VINTAGE  CRYSTAL EARRINGS 
 ###################